In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import sys

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

# Add path to the nxsdk_modules package.
sys.path.append('/homes/dcicch/models')
sys.path.append('/homes/dcicch/NxSDK_Package/nxsdk-apps')


# Enable SLURM to run network on Loihi.
os.environ['SLURM'] = '1'
os.environ['PYTHONUNBUFFERED'] = '1'

os.environ["PARTITION"] = "nahuku32_2h"
os.environ["BOARD"] = 'ncl-ext-ghrd-01'

snipDir = os.path.abspath(os.path.join('..', 'snips', 'reset_model_states'))


In [9]:
import os
import keras
#from keras.models import Model
from keras.layers import Dropout, Flatten, Conv2D, Input, Dense

import tensorflow as tf
from tensorflow import keras

from abc import ABC
from collections import namedtuple

from nxsdk_modules_ncl.dnn.src.utils import extract
from nxsdk_modules_ncl.dnn.src.dnn_layers import NxInputLayer, NxConv2D, NxDense, \
    NxModel, NxFlatten, ProbableStates

from nxsdk_modules_ncl.dnn.src.optimization import ExclusionCriteria
from nxsdk_modules_ncl.dnn.composable.composable_dnn import ComposableDNN

from nxsdk_modules_ncl.dnn.src.utils import to_integer

from nxsdk_modules_ncl.dnn.composable.composable_dnn import ComposableDNN as DNN
from nxsdk_modules.input_generator.input_generator import InputGenerator

from nxsdk.composable.model import Model
from nxsdk.logutils.nxlogging import set_verbosity,LoggingLevel
import csv
#from nxsdk.api.n2a as 
set_verbosity(LoggingLevel.ERROR)

In [10]:
batch_size = 32
num_training_epochs = 2
input_shape = (28, 28, 1)

# set later
#num_steps_per_img = 75
num_train_samples = 60000
num_test_samples = 128

# EnergyProbes allow to profile execution time, power and thus energy consumption
enable_energy_probe = True
# set later
#execution_time_probe_bin_size = 75 # Small numbers slows down execution

# Not yet supported
measure_accuracy_runtime_trade_off = False # Not yet supported
runtimes = [128, 256, 512, 1024, 2048]

In [11]:
from keras.datasets import mnist
from keras.utils import np_utils

# Load standard MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize input so we can train ANN with it. 
# Will be converted back to integers for SNN layer.
x_train = x_train[:num_train_samples, :, :] / 255
x_test = x_test / 255

# Add a channel dimension.
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# One-hot encode target vectors.
y_train = np_utils.to_categorical(y_train[:num_train_samples], 10)
y_test = np_utils.to_categorical(y_test, 10)

In [12]:
def runModel(num_steps_per_sample, x_test, y_test):
    """Runs the SNN Model to classify test images."""
      
    # Initialize arrays for results
    num_samples = len(y_test)
    classifications = np.zeros(num_samples, int)
    labels = np.zeros(num_samples, int)

    # Run DNN to classify images
    tStart = time.time()
    snn_model.run(num_steps_per_sample * num_samples, aSync=True)
    tEndBoot = time.time()

    # Enqueue images by pushing them into InputGenerator
    print("Queuing images...")
    for i, (x, y) in enumerate(zip((x_test* 255).astype(int), y_test)):
        input_generator.encode(x)
        labels[i] = np.argmax(y)
    tEndInput = time.time()

    # Read out classification results for all images
    print("Waiting for classification to finish...")
    classifications = list(dnn.readout_channel.read(num_samples))
    tEndClassification = time.time()
    
    # finishRun fetches EnergyProbe data if configured
    snn_model.finishRun()
    
    return tStart, tEndBoot, tEndInput, tEndClassification, classifications, labels

def calcAccuracy(classifications, labels):
    """Computes classification accuracy for a set of images given classification and labels."""
    errors = classifications != labels
    num_errors = np.sum(errors)
    num_samples = len(classifications)
    return (num_samples-num_errors)/num_samples

In [13]:
def plot_samples(samples, guesses, lables, numCols=5, size=28):
    """Plots samples as an array of images."""
    
    import math
    numSamples = len(samples)
    numRows = int(math.ceil(numSamples/numCols))
    plt.figure(3, figsize=(20, 10))
    i = 0
    for c in range(numCols):
        for r in range(numRows):
            plt.subplot(numRows, numCols, i+1)
            plt.imshow(np.reshape(samples[i,:], (size, size)))
            plt.axis('off')
            plt.title('C:{}/L:{}'.format(guesses[i], lables[i]))
            i += 1
            if i == numSamples:
                break
        if i == numSamples:
            break
    plt.show()

In [14]:
# utils
def batch_data(data, batch_size):
    data_size, n_features = data.shape
    n_batches = int(data_size / batch_size)
    batched_data = np.zeros((n_batches, batch_size, n_features))

    for i in range(n_batches):
        start = batch_size * i
        batched_data[i] = data[start: start + batch_size]
    return batched_data


def derivative_relu(x, alpha=0.1):
    return tf.where(x < 0, alpha, 1.0)

def copi_backpropagation(**kwargs):
    """ returns all deltas for dloss/da_l """
    df = kwargs['df']
    acts = kwargs['acts']
    ys = kwargs['ys']
    y_hat = kwargs['y_hat']
    alpha = kwargs['alpha_relu']
    layers = kwargs['layers']

    d = -df(acts[-1], alpha) * (ys[-1] - y_hat)
    deltas = [d]
    for i in reversed(range(0, len(layers) - 1)):
        W = layers[i + 1].W
        R = layers[i + 1].R

        d = (d @ tf.transpose(W) @ tf.transpose(R)) * df(acts[i], alpha)
        deltas.append(d)
    return reversed(deltas)


def copi_broadcast_alignment(**kwargs):
    """ returns all deltas for dloss/da_l using a random feedback matrix B """
    ys = kwargs['ys']
    y_hat = kwargs['y_hat']
    layers = kwargs['layers']

    error = y_hat - ys[-1]

    deltas = []
    for layer in layers[:-1]:
        feedback = error @ layer.B
        deltas.append(feedback)
    deltas.append(error)

    return deltas

In [15]:
# layer
InternalState = namedtuple('InternalState', ('u', 'r', 'v'))


class LIF(ABC, keras.layers.Layer):
    def __init__(self, n_neurons: int, n_err: int, rest: float = 0.0, threshold: float = 0.4, refractory: float = 1.0,
                 tau: float = 20.0, dt: float = 0.25, epsilon: float = 0.001,
                 train_fn_name=None, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.n_neurons = n_neurons
        self.resting_potential = tf.cast(rest, self.dtype)
        self.threshold = tf.cast(threshold, self.dtype)
        self.refractory = tf.cast(refractory, self.dtype)
        self.tau = tf.cast(tau, self.dtype)
        self.eps = tf.cast(epsilon, self.dtype)
        self.dt = tf.cast(dt, self.dtype)

        self.W = None
        self.b = None
        if train_fn_name == 'ba':
            self.B = self.add_weight('B', shape=(n_err, n_neurons), initializer='GlorotUniform', trainable=False)
        self.state = None
        self.inputs = None
        self.alpha = tf.exp(-self.dt / self.tau)

    def build(self, input_shape: list) -> None:
        batch_size, n_inputs = input_shape
        self.inputs = tf.zeros(n_inputs, dtype=self.dtype)

        self.W = self.add_weight('W', shape=(n_inputs, self.n_neurons), initializer='GlorotUniform', trainable=True)
        self.b = self.add_weight('b', shape=(self.n_neurons,), initializer='GlorotUniform', trainable=True)
        self.state = self.zero_state(batch_size)

    def zero_state(self, batch_size: int) -> InternalState:
        return InternalState(
            u=tf.fill((batch_size, self.n_neurons), tf.cast(self.resting_potential, dtype=self.dtype)),
            r=tf.zeros((batch_size, self.n_neurons), self.dtype),
            v=tf.zeros((batch_size, self.n_neurons), self.dtype)
        )

    def call(self, inputs: tf.Tensor, *args, **kwargs) -> tf.Tensor:
        """
        simulate one time-step, updates internal values and returns whether a spike occurred or not

        :param inputs: one dimensional tensor of inputs with size n_in
        :return: boolean tensor of spike or non-spike values
        """

        # old internal state
        u, r, _ = self.state

        v = inputs @ self.W + self.b
        new_r = tf.where(r > 0., r + self.dt, r)
        new_r = tf.where(new_r > self.refractory, 0., new_r)
        new_u = tf.where(new_r > 0., self.resting_potential, u + (v - u) * (self.dt / self.tau))
        new_r = tf.where(new_u > self.threshold, self.eps, new_r)

        # new observable state
        z = tf.cast(tf.where(new_r > 0, 1.0, 0.0), self.dtype)

        # new internal state
        self.state = InternalState(new_u, new_r, v)
        self.inputs = inputs

        return z

    def reset(self, batch_size: int):
        self.state = self.zero_state(batch_size)

In [16]:
# network
class CopiLayer(keras.layers.Layer):
    def __init__(self, units, activation, n_err=None, *args, **kwargs):
        super(CopiLayer, self).__init__(*args, **kwargs)
        self.n = units
        self.activation = activation
        if n_err is not None:
            self.B = self.add_weight(name='W', shape=(n_err, self.n),
                                     initializer='GlorotNormal',
                                     trainable=False)

        self.W = None
        self.R = None

    def build(self, input_shape):
        _, n_in = input_shape

        self.W = self.add_weight(name='W', shape=(n_in, self.n), initializer='GlorotNormal', trainable=True)
        self.R = self.add_weight(name='R', shape=(n_in, n_in), initializer='Identity', trainable=True)

    def call(self, y_prev, *args, **kwargs):
        x = y_prev @ self.R
        a = x @ self.W
        y = self.activation(a)
        return x, a, y


class CopiNetwork(keras.Model):
    def __init__(self, units, activations, lr_w, lr_r, df, alpha, rule='bp', *args, **kwargs):
        super(CopiNetwork, self).__init__(*args, **kwargs)
        n_err = None
        if rule == 'bp':
            self.rule = copi_backpropagation
        elif rule == 'ba':
            self.rule = copi_broadcast_alignment
            n_err = units[-1]
        else:
            print(f"rule not supported yet; choose between 'ba' and 'bp'")

        self.ls = [CopiLayer(u, a, n_err) for u, a in zip(units, activations)]
        self.df = df
        self.alpha = alpha
        self.lr_w = lr_w
        self.lr_r = lr_r
        if rule == 'bp':
            self.rule = copi_backpropagation
        elif rule == 'ba':
            self.rule = copi_broadcast_alignment
        else:
            print(f"rule not supported yet; choose between 'ba' and 'bp'")

    def call(self, inputs, training=None, mask=None, **kwargs):
        xs, acts, ys = [], [], []

        for layer in self.ls:
            x, a, y = layer(inputs)
            inputs = y
            xs.append(x)
            acts.append(a)
            ys.append(y)

        return xs, acts, ys if training else ys

    def train_step(self, data, *args, **kwargs):
        inputs, y_true = data
        batch_size, feature_size = inputs.shape

        # forward pass
        xs, acts, ys = self(inputs, True)

        # backward pass
        rule_info_dict = {
            'df': self.df,
            'acts': acts,
            'ys': ys,
            'y_hat': y_true,
            'alpha_relu': self.alpha,
            'layers': self.ls
        }

        deltas = self.rule(**rule_info_dict)

        # mixed update pass
        for x, a, delta, layer in zip(xs, acts, deltas, self.ls):
            W, R = layer.trainable_variables

            z = a + delta
            q = x @ R
            xT = tf.transpose(x)
            squares = (xT @ x) * tf.eye(x.shape[-1])  # diag(E[x^2])

            dW = (xT @ z - squares @ W) / batch_size
            dR = (xT @ q - squares @ R) / batch_size

            layer.W.assign_add(self.lr_w * dW)
            layer.R.assign_sub(self.lr_r * dR)

        self.compiled_metrics.update_state(y_true, ys[-1])
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        x, y = data
        ys = self(x)[-1]

        self.compiled_metrics.update_state(y, ys[-1])
        return {m.name: m.result() for m in self.metrics}


In [33]:
## timing ANN
from timeit import default_timer as timer

class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_test_begin(self, logs={}):
        self.starttime = timer()
    def on_test_end(self, logs={}):
        self.logs.append(timer()-self.starttime)

train_model = True

(train_X, train_y), (test_X, test_y) = tf.keras.datasets.mnist.load_data(
    path='mnist.npz')
train_X = train_X.reshape(-1, 784) / 255
train_y = tf.one_hot(train_y, depth=10, axis=1)
test_X = test_X.reshape(-1, 784) / 255
test_y = tf.one_hot(test_y, depth=10, axis=1)
     
## Array for time
## Array for accuracy
## Array for Energy
vals_dense = np.zeros((5, 5, 10))
    
    
# Path for pre-trained model
pretrained_model_path = os.path.join(os.path.abspath(''),
                                     'models', 
                                     'a_minist_model.h5')

layer_sizes = [10, 40, 100, 400, 1000]
step_sizes = [30, 50, 100]

ann_time = np.zeros((len(layer_sizes), 10))
accuracy_ANN = np.zeros((len(layer_sizes), 10))
vals_dense = np.zeros((5, len(layer_sizes), len(step_sizes), 10))

mean_vals_t1 = np.zeros((len(layer_sizes), len(step_sizes)))
std_vals_t1 = np.zeros((len(layer_sizes), len(step_sizes)))
mean_vals_t2 = np.zeros((len(layer_sizes), len(step_sizes)))
std_vals_t2 = np.zeros((len(layer_sizes), len(step_sizes)))
mean_vals_acc = np.zeros((len(layer_sizes), len(step_sizes)))
std_vals_acc = np.zeros((len(layer_sizes), len(step_sizes)))
mean_vals_e1 = np.zeros((len(layer_sizes), len(step_sizes)))
std_vals_e1 = np.zeros((len(layer_sizes), len(step_sizes)))
mean_vals_e2 = np.zeros((len(layer_sizes), len(step_sizes)))
std_vals_e2 = np.zeros((len(layer_sizes), len(step_sizes)))


n_dim = 10
    
# Generate model
for k in range(len(step_sizes)):
    print(f"Number of time steps: {step_sizes[k]}")
    num_steps_per_img = step_sizes[k]
    execution_time_probe_bin_size = step_sizes[k]
    for i in range(len(layer_sizes)):
        print(f"Neurons in hidden layer: {layer_sizes[i]}")
        for j in range(10):
            if train_model or not os.path.isfile(pretrained_model_path):
                # set all variable accordingly
#                 num_steps_per_img = 100  # timesteps per image
                layers = [layer_sizes[i], 10]  # neurons per layer
                units = [layer_size, 10]

#                 training_logger = keras.callbacks.CSVLogger('train_log.csv', separator=",", append=True)
                net = CopiNetwork(units, activations,
                              lr_w=1e-2, lr_r=1e-2,
                              df=derivative_relu,
                              alpha=alpha,
                              rule='bp')
                net.compile(optimizer='adam', loss='mse', metrics=['categorical_accuracy', 'mse'])
                net.load_weights('weights/weights-time_step{}-layer_size{}-n{}-sg'.format(num_steps_per_img, layer_sizes[i], n))

#                 net.fit(train_X, train_y, epochs=10, batch_size=1000, callbacks=[training_logger])
#                 print('evaluation mse, accuracy', net.evaluate(test_X, test_y))


            vth_mant = 2**9
            bias_exp = 6
            weight_exponent = 0
            synapse_encoding = 'sparse'
            #exclusionCriteria = ExclusionCriteria()

            inputLayer = NxInputLayer((28,28), 
                                         vThMant=vth_mant, 
                                         biasExp=bias_exp)

            input_layer = NxFlatten()(inputLayer.input)

            layer = NxDense(layer_sizes[i], activation='relu')(input_layer)

            outputs = NxDense(10, activation='softmax')(layer)

            snn_nxmodel = NxModel(inputLayer.input, outputs, numCandidatesToCompute=1)

            #snn_nxmodel.summary()


            # Extract weights and biases from parameter list.
            parameters = net.get_weights()
            weights = parameters[0::2]
            biases = parameters[1::2]

            # Quantize weights and biases using max-normalization (Strong quantization loss if distributions have large tails)
            parameters_int = []
            for w, b in zip(weights, biases):
                w_int, b_int = to_integer(w, b, 8)
                parameters_int += [w_int, b_int]

            # Set quantized weigths and biases for spiking model
            snn_nxmodel.set_weights(parameters_int)


            # NxModel is not yet implemented as a Composable -> Wrap it with DNN composable class
            dnn = DNN(model=snn_nxmodel, num_steps_per_img=num_steps_per_img)

            input_generator = InputGenerator(shape=(28,28), interval=num_steps_per_img)

            input_generator.setBiasExp(bias_exp)


            #set_verbosity(LoggingLevel.ERROR)

            # Initialize empty model
            snn_model = Model("dnn_model")

            # Add DNN and InputGenerator to empty model
            snn_model.add(dnn)
            snn_model.add(input_generator)


            # Connect InputGenerator to DNN
            # (Explicit)
            # input_generator.ports.output.connect(dnn.ports.input)
            # (Implicit when ports can be inferred)
            input_generator.connect(dnn)

            # Enfore particular execution order or processes/snips executing in the same phase
            # (Here: Execute input injection as bias currents after network reset)
            input_generator.processes.inputEncoder.executeAfter(dnn.processes.reset)

            snn_model.compile()

            if enable_energy_probe:
                from nxsdk.api.enums.api_enums import ProbeParameter
                from nxsdk.graph.monitor.probes import PerformanceProbeCondition
                eProbe = snn_model.board.probe(probeType=ProbeParameter.ENERGY, 
                                               probeCondition=PerformanceProbeCondition(
                                                    tStart=1, 
                                                    tEnd=num_test_samples*num_steps_per_img, 
                                                    bufferSize=1024, 
                                                    binSize=execution_time_probe_bin_size))

            snn_model.start(snn_nxmodel.board)

            tStart, tEndBoot, tEndInput, tEndClassification, classifications, labels = runModel(num_steps_per_img, 
                                                                                                x_test[:num_test_samples, :, :, 0], 
                                                                                                y_test[:num_test_samples])

             # Runtime statistics
            runtimeBoot = tEndBoot-tStart
            runtimeInput = tEndInput-tStart
            runtimeClassification = tEndClassification-tStart

            vals_dense[0, i, k, j] = runtimeClassification-runtimeInput
            vals_dense[3, i, k, j] = np.mean(eProbe.spikingTimePerTimeStep)*num_steps_per_img/1e3

            # Accuracy statistics
            errors = classifications != labels
            num_errors = np.sum(errors)

            vals_dense[1, i, k, j] = (num_test_samples-num_errors)/num_test_samples


            vals_dense[2, i, k, j] = eProbe.totalEnergy
            vals_dense[4, i, k, j] = np.mean(eProbe.totalEnergyPerTimeStep)*num_steps_per_img

            if not measure_accuracy_runtime_trade_off:
                snn_model.disconnect()

    mean_vals_t1[:, k] = np.mean(vals_dense[0, :, k, :], 1)
    std_vals_t1[:, k] = np.std(vals_dense[0, :, k, :], 1)

    mean_vals_t2[:, k] = np.mean(vals_dense[3, :, k, :], 1)
    std_vals_t2[:, k] = np.std(vals_dense[3, :, k, :], 1)

    mean_vals_acc[:, k] = np.mean(vals_dense[1, :, k, :], 1)
    std_vals_acc[:, k] = np.std(vals_dense[1, :, k, :], 1)

    mean_vals_e1[:, k] = np.mean(vals_dense[2, :, k, :], 1)
    std_vals_e1[:, k] = np.std(vals_dense[2, :, k, :], 1)

    mean_vals_e2[:, k] = np.mean(vals_dense[4, :, k, :], 1)
    std_vals_e2[:, k] = np.std(vals_dense[4, :, k, :], 1)

total_list = [mean_vals_t1, std_vals_t1, mean_vals_t2, std_vals_t2, mean_vals_acc, 
              std_vals_acc, mean_vals_e1, std_vals_e1, mean_vals_e2, std_vals_e2]
csv_names = ["spike_mean_vals_t1.csv", "spike_std_vals_t1.csv", "spike_mean_vals_t2.csv", 'spike_std_vals_t2.csv',
             'spike_mean_vals_acc.csv', 'spike_std_vals_acc.csv', 'spike_mean_vals_e1.csv', 'spike_std_vals_e1.csv',
             'spike_mean_vals_e2.csv', 'spike_std_vals_e2.csv']
for i in range(len(csv_names)):
    with open(csv_names[i], mode='w') as employee_file:
        employee_writer = csv.writer(employee_file, delimiter=',',  quotechar='"', 
                                     quoting=csv.QUOTE_MINIMAL)
        employee_writer.writerow(total_list[i])

Number of time steps: 10
Neurons in hidden layer: 10
Epoch 1/2
60/60 [==============================] - 115s 2s/step - mse: 0.0372 - accuracy: 0.7470
Epoch 2/2
313/313 [==============================] - 62s 197ms/step - mse: 0.0217 - accuracy: 0.8821
evaluation mse, accuracy [0.021712832152843475, 0.882099986076355]
x

x

x

x

Queuing images...spikes... . .. 
Waiting for classification to finish...
Epoch 1/2ssing timeseries... 
 9/60 [===>..........................] - ETA: 1:43 - mse: 0.0666 - accuracy: 0.4676

KeyboardInterrupt: 

In [19]:
print(parameters_int)

[array([[-1, -1, -1, ...,  0,  0,  0],
       [ 0,  0, -1, ...,  0,  0,  0],
       [ 0,  1, -1, ...,  0,  0,  0],
       ...,
       [ 0,  0, -1, ...,  0, -1,  1],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  1,  0, ...,  1,  0,  0]]), array([-28,  57,   5, -15,  34,  73, -23,  55, -71, -22])]


In [ ]:
fig = plt.figure(figsize=(10,5))

for i in range(len(step_sizes)):
    plt.errorbar(layer_sizes, mean_vals_t1[:, i], yerr = std_vals_t1[:, i],
                 label = f"step_sizes = {step_sizes[i]}")
    plt.ylabel("time [s]")
    plt.xlabel("Layers")
    plt.title("Total Time over hidden layers")
    plt.xscale('log')
    plt.legend()
    

fig.savefig('total_time_dense_layer_stepsize.jpg', bbox_inches='tight', dpi=150)

plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5))

for i in range(len(step_sizes)):
    plt.errorbar(layer_sizes, mean_vals_e1[:, i], yerr = std_vals_e1[:, i],
                label = f"step_sizes = {step_sizes[i]}")
    plt.xlabel("Hidden Layer size")
    plt.ylabel("energy [µJ]")
    plt.title("Total Energy over hidden layers")
    plt.xscale('log')
    plt.legend()
    
fig.savefig('total_energy_dense_layer_stepsize.jpg', bbox_inches='tight', dpi=150)

plt.show()

In [15]:
fig = plt.figure(figsize=(10,5))

for i in range(len(step_sizes)):
    plt.errorbar(layer_sizes, mean_vals_acc[:, i], yerr = std_vals_acc[:, i],
                label = f"step_sizes = {step_sizes[i]}")
    plt.ylabel("Accuracy")
    plt.xlabel("Hidden Layer size")
    plt.title("Accuracy over hidden layers")
    plt.xscale('log')

fig.savefig('accuracy_dense_layer_stepsize.jpg', bbox_inches='tight', dpi=150)
plt.show()

In [34]:
fig = plt.figure(figsize=(10,5))

for i in range(len(step_sizes)):
    plt.errorbar(layer_sizes, mean_vals_t2[:, i], yerr = std_vals_t2[:, i],
                 label = f"step_sizes = {step_sizes[i]}")
    plt.xlabel("Hidden Layer size")
    plt.ylabel("Time [ms]")
    plt.title("Time per time step over hidden layers")
    plt.xscale('log')
    plt.legend()
    

fig.savefig('time_timestep_av_dense_layer_stepsize.jpg', bbox_inches='tight', dpi=150)

plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5))

for i in range(len(step_sizes)):
    plt.errorbar(layer_sizes, mean_vals_e2[:, i], yerr = std_vals_e2[:, i],
                label = f"step_sizes = {step_sizes[i]}")
    plt.xlabel("Hidden Layer size")
    plt.ylabel("Energy [µJ]")
    plt.title("Energy per time step over hidden layers")
    plt.xscale('log')
    plt.legend()
    
fig.savefig('energy_timestep_av_dense_layer_stepsize.jpg', bbox_inches='tight', dpi=150)

plt.show()

In [12]:
mean_time_ann = np.mean(ann_time, axis = 1)
sd_time_ann = np.std(ann_time, axis = 1)

mean_acc_ann = np.mean(accuracy_ANN, axis = 1)
sd_acc_ann = np.std(accuracy_ANN, axis = 1)

fig = plt.figure(figsize=(10,5))

plt.plot(layer_sizes, mean_time_ann)
plt.errorbar(layer_sizes, mean_time_ann, yerr = sd_time_ann, fmt ='o', mfc = 'b', mec = 'b', ms = 2)
plt.ylabel("time[s]")
plt.xlabel("Hidden Layer size")
plt.xscale('log')

fig.savefig('ANN_time_dense.jpg', bbox_inches='tight', dpi=150)
plt.show()

In [13]:
fig = plt.figure(figsize=(10,5))

plt.plot(layer_sizes, mean_acc_ann)
plt.errorbar(layer_sizes, mean_acc_ann, yerr = sd_acc_ann, fmt ='o', mfc = 'b', mec = 'b', ms = 2)
plt.ylabel("accuracy")
plt.xlabel("Hidden Layer size")
plt.xscale('log')

fig.savefig('ANN_acc_dense.jpg', bbox_inches='tight', dpi=150)
plt.show()

In [14]:
## Throughput values 
through_time = 1/(mean_vals_t1/num_test_samples)
through_time_sd = 1/(std_vals_t1/(num_test_samples)**2)

through_energy = (mean_vals_e1/num_test_samples)
through_energy_sd = 1/(std_vals_e1/(num_test_samples)**2)

In [ ]:
fig = plt.figure(figsize=(10,5))

for i in range(len(step_sizes)):
    plt.errorbar(layer_sizes, through_time[:, i], yerr = through_time_sd[:, i],
                 label = f"step_sizes = {step_sizes[i]}")
    plt.xlabel("Hidden Layer size")
    plt.ylabel("Inferences per Second")
    plt.title("Inference Time over hidden layer size")
    plt.xscale('log')
    plt.legend()
    

fig.savefig('Inference_time_dense_layer_stepsize.jpg', bbox_inches='tight', dpi=150)

plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5))

for i in range(len(step_sizes)):
    plt.errorbar(layer_sizes, through_energy[:, i], yerr = through_energy_sd[:, i],
                label = f"step_sizes = {step_sizes[i]}")
    plt.xlabel("Hidden Layer size")
    plt.ylabel("Energy per Inference [µJ]")
    plt.title("Inference Energy over hidden layer size")
    plt.xscale('log')
    plt.legend()
    
fig.savefig('Inference_energy_dense_layer_stepsize.jpg', bbox_inches='tight', dpi=150)

plt.show()